In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = list(map(lambda s : list(idxs).index(s), a0))
    a1 = list(map(lambda s : list(idxs).index(s), a1))
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [4]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
import pickle

lang_1 = pickle.load(open('data_beer_lemm_90K_sep.p', 'rb'))

In [13]:
data_train = lang_1['training']
lang_1['vocab_size'] = len(lang_1['word2idx'])
lang_1['word_dim'] = 200
import torch
import model
mod = model.Model(lang_1=lang_1, num_aspect=20)
mod.load_values('results/model_SatFeb1716:51:402018_beer_90K_lemm')

In [31]:
embeds = []
for i, doc in enumerate(lang_1['training']) :
    if i in val_idxs :
        new_sent = [sent for sent in doc if len(sent) > 0]
        mod.train_batch(new_sent, [new_sent], update=False)
        embeds.append(mod.z_s)

In [32]:
for i, e in enumerate(embeds) :
    embeds[i] = e.mean(dim=0).data.cpu().numpy()
e = np.array(embeds)

In [33]:
from sklearn.preprocessing import normalize
e = normalize(e, 'l2')
scores = np.dot(e, e.T)
nb_studies = len(val_idxs)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [34]:
aucs = [0] * len(val_idxs)
aspects = [str(i) for i in range(4)]
from sklearn.metrics import roc_auc_score
for aspect_j in aspects :
    for i in range(len(val_idxs)) :
        aucs[i] = roc_auc_score(H[aspect_j][i], scores[i])
    print(aspect_j, np.mean(aucs))

0 0.5001488836063909
1 0.499976687348609
2 0.4998983137428305
3 0.499860037330679
